In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)


data = pd.read_excel('../data/nba_player_data.xlsx')

In [3]:
data.sample(5)

,Year,Season_type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
1503,2014-15,Regular%20Season,203903,142,Jordan Clarkson,1610612747,LAL,59,1476,267,596,0.448,38,121,0.314,131,158,0.829,56,135,191,206,51,12,96,109,703,711,2.15,0.53
6492,2021-22,Regular%20Season,203944,22,Julius Randle,1610612752,NYK,72,2544,512,1246,0.411,120,390,0.308,303,401,0.756,125,591,716,370,53,39,245,205,1447,1548,1.51,0.22
241,2012-13,Regular%20Season,203085,242,Austin Rivers,1610612740,NOH,61,1418,145,390,0.372,28,86,0.326,59,108,0.546,16,93,109,127,26,9,74,122,377,280,1.72,0.35
4529,2018-19,Regular%20Season,203920,325,Khem Birch,1610612753,ORL,50,643,91,151,0.603,0,1,0.000,58,83,0.699,79,111,190,38,18,29,20,72,240,410,1.90,0.90
8235,2023-24,Regular%20Season,1631204,187,Marcus Sasser,1610612765,DET,71,1351,219,512,0.428,90,240,0.375,58,66,0.879,11,114,125,236,44,12,90,103,586,612,2.62,0.49


In [4]:
data.shape

(8835, 30)

Data cleaning & analysis preparation

In [5]:
data.drop(columns=['RANK', 'EFF'], inplace=True)

In [6]:
data['season_start_year'] = data['Year'].str[:4].astype(int)

In [7]:
data['TEAM'].replace(to_replace=['NOP', 'NOH'], value='NO', inplace=True) # use 'NO' for New Orleans
data.TEAM.nunique() # there should be 30 unique teams

C:\Users\delya\AppData\Local\Temp\ipykernel_18316\582994501.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['TEAM'].replace(to_replace=['NOP', 'NOH'], value='NO', inplace=True) # use 'NO' for New Orleans


30

In [ ]:
data['Season_type'].replace('Regular%20Season', 'Regular', inplace=True)

In [9]:
rs_df = data[data['Season_type']=='Regular']
playoffs_df = data[data['Season_type']=='Playoffs']

In [10]:
data.columns

Index(['Year', 'Season_type', 'PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'AST_TOV', 'STL_TOV', 'season_start_year'],
      dtype='object')

In [11]:
total_cols = ['MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA',
    'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',]

Which player stats are correlated with each other?

In [39]:
data_per_min = data.groupby(['PLAYER', 'PLAYER_ID', 'Year'])[total_cols].sum().reset_index()
for col in data_per_min.columns[4:]:
    data_per_min[col] = data_per_min[col] / data_per_min['MIN']

data_per_min['FG%'] = data_per_min['FGM'] / data_per_min['FGA']
data_per_min['3PT%'] = data_per_min['FG3M'] / data_per_min['FG3A']
data_per_min['FT%'] = data_per_min['FTM'] / data_per_min['FTA']
data_per_min['FG3A%'] = data_per_min['FG3A'] / data_per_min['FGA']
data_per_min['PTS/FGA'] = data_per_min['PTS'] / data_per_min['FGA']
data_per_min['FG3M/FGM'] = data_per_min['FG3M'] / data_per_min['FGM']
data_per_min['FTA/FGA'] = data_per_min['FTA'] / data_per_min['FGA']
data_per_min['TRU%'] = 0.5 * data_per_min['PTS'] / (data_per_min['FGA'] + 0.475 * data_per_min['FTA'])
data_per_min['AST_TOV'] = data_per_min['AST'] / data_per_min['TOV']

# Only look at players who have recorded at least 50 min of play time in a season
data_per_min = data_per_min[data_per_min['MIN']>=50]
data_per_min.drop(columns='PLAYER_ID', inplace=True)
# data_per_min.head(4)

fig = px.imshow(data_per_min.select_dtypes(include=[float, int]).corr(), width=640, height=640)
fig.show()

---